# Обучение модели TabNetL

In [1]:
!pip install pytorch-tabnet neptune-client

     |████████████████████████████████| 90 kB 2.8 MB/s 
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 67 kB 3.0 MB/s 
     |████████████████████████████████| 138 kB 10.6 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.4.117-py2.py3-none-any.whl size=149569 sha256=a289026ac0d5921266fdda6f8dca14d124ab4299a9aedca5adc35d4c4f07411d
  Stored in directory: /root/.cache/pip/wheels/c2/ed/91/3ee4201218bc6087407bd61d751452c228bda95f41867ec036
  Created wheel for msgpack-python: filename=msgpack_python-0.5.6-cp37-cp37m-linux_x86_64.whl size=302585 sha256=9a5dc0a2a1e9f87bb2537f9410f09800f079f3e0d52642db48c3a197688f2ed3
  Stored in directory: /root/.cache/pip/wheels/f8/6c/02/92ebc97f3b99ad5bfc675be2c513f9cb3504fdbe338314f377
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-py3-none-any.whl size=18119 sha256=b21cb5ff749399bdf0bc3e91272cccecbed31a1e62117d84fcb0edc40704e256
  Stored in directory: /root/.cache/pip/wheel

In [2]:
import os
import numpy as np
import torch
import random
import pickle

from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
import neptune

In [3]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
os.listdir('../input/')

['experience2id.pickle',
 'names.npy',
 'employment_label.npy',
 'schedule_label.npy',
 'city2id.pickle',
 'salary_scaled.npy',
 'employment2id.pickle',
 'salary_orig.npy',
 'schedule2id.pickle',
 'city_label.npy',
 'experience_label.npy']

Загрузем предобработанные данные.

Объединим данные в единый массив и разобъём всё на тестовую и валидационную выборки.

In [5]:
salary = np.load('../input/salary_scaled.npy')

In [6]:
salary = salary.reshape(-1, 1)

Объединим данные в единый массив и разобъём всё на тестовую и валидационную выборки.[](http://)

In [7]:
X = np.concatenate((
    np.load('../input/names.npy'),
    np.load('../input/experience_label.npy'),
    np.load('../input/schedule_label.npy'),
    np.load('../input/employment_label.npy'),
    np.load('../input/city_label.npy')
), axis=1)

In [8]:
len(X) * 0.8

1098592.0

Зададим количество параметров для того, чтобы наша модель смогла создать эмбеддинги

In [9]:
with open('../input/experience2id.pickle', 'rb') as handle:
    experience2id = pickle.load(handle)
with open('../input/schedule2id.pickle', 'rb') as handle:
    schedule2id = pickle.load(handle)
with open('../input/employment2id.pickle', 'rb') as handle:
    employment2id = pickle.load(handle)
with open('../input/city2id.pickle', 'rb') as handle:
    city2id = pickle.load(handle)

In [10]:
cat_dims = [len(experience2id), len(schedule2id), len(employment2id), len(city2id),]

Инициализируем Нептун для отслеживания состояния системы и сохранения результата


In [11]:
PARAMS = {
    'description': 'TabNetL'
}
neptune.init('blanchefort/salary', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiZDIxYzYxODctMWU1Zi00ZDIzLWJkYjEtYmNlZDQxMTUxZjA4In0=')
neptune.create_experiment(name='SalaryTabNetL2',params=PARAMS)

https://ui.neptune.ai/blanchefort/salary/e/SAL-36


Experiment(SAL-36)

Инициализируем модель

In [12]:
clf = TabNetRegressor(
    verbose=10,
    optimizer_fn=torch.optim.AdamW,
    optimizer_params=dict(lr=2e-2),
    scheduler_params = {"gamma": 0.95,
                     "step_size": 20},
    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
    seed=seed,
    clip_value=2.,
    cat_idxs=[768, 769, 770, 771],
    cat_dims=cat_dims,
    cat_emb_dim=[1, 1, 1, 100],

    n_d=64,
    n_a=64,
    n_steps=5,
    gamma=1.5,
    n_independent=2,
    n_shared=2,
    lambda_sparse=1e-4,
    momentum=0.3,
    
)

Device used : cuda


Запускаем обучение

In [13]:
clf.fit(
    X[:1098592],
    salary[:1098592],
    X[1098592:],
    salary[1098592:],
    max_epochs=3000,
    loss_fn=torch.nn.L1Loss(),
    batch_size=16384,
    virtual_batch_size=256,
    patience=200,
    drop_last=False,
    )

Will train until validation stopping metric hasn't improved in 200 rounds.
---------------------------------------
| EPOCH |  train  |   valid  | total time (s)
| 10    | -0.85917 |  -0.90205 |   676.4     
| 20    | -0.83118 |  -0.88598 |   1347.9    
| 30    | -0.83381 |  -0.89212 |   2020.0    
| 40    | -0.82988 |  -0.89940 |   2687.2    
| 50    | -0.73502 |  -0.79886 |   3354.2    
| 60    | -0.71815 |  -0.75500 |   4021.5    
| 70    | -0.71617 |  -0.76401 |   4685.4    
| 80    | -0.71446 |  -0.76072 |   5347.0    
| 90    | -0.71359 |  -0.75997 |   6010.1    
| 100   | -0.70756 |  -0.75810 |   6677.7    
| 110   | -0.70444 |  -0.76089 |   7354.2    
| 120   | -0.70139 |  -0.75669 |   8032.8    
| 130   | -0.69840 |  -0.75704 |   8711.0    
| 140   | -0.69179 |  -0.75711 |   9383.4    
| 150   | -0.68460 |  -0.76974 |   10059.1   
| 160   | -0.67796 |  -0.75627 |   10735.0   
| 170   | -0.66632 |  -0.76719 |   11411.3   
| 180   | -0.65909 |  -0.76401 |   12090.4   
| 190   | -

Сохраним полученный результат для дальнейшего использования

In [14]:
with open('SalaryTabnet_L.pt', 'wb') as model_file:
    torch.save(clf, model_file)
neptune.log_artifact('SalaryTabnet_L.pt', 'SalaryTabnetL_GPU.pt')

device = torch.device('cpu')
clf.network.to(device)
with open('SalaryTabnet_L.pt', 'wb') as model_file:
    torch.save(clf, model_file)
neptune.log_artifact('SalaryTabnet_L.pt', 'SalaryTabnetL_CPU.pt')